In [1]:
import numpy as np
import pandas as pd 
import os
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
import tensorflow.keras
import time
import imutils
import hashlib

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, AveragePooling2D, Dense, Activation, Dropout, Flatten, Input
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import *
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.applications import MobileNetV2
from typing import List
from pathlib import Path
from imutils.video import VideoStream
from PIL import Image

In [2]:
label_map = ['Anger', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

## Transfer Learning on MobileNet-V2

In [23]:
image_generator = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    zca_epsilon=1e-06,  # epsilon for ZCA whitening
    rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
    # randomly shift images horizontally (fraction of total width)
    width_shift_range=0.1,
    # randomly shift images vertically (fraction of total height)
    height_shift_range=0.1,
    shear_range=0.1,  # set range for random shear
    zoom_range=0.0,  # set range for random zoom
    channel_shift_range=0.0,  # set range for random channel shifts
    # set mode for filling points outside the input boundaries
    fill_mode="nearest",
    cval=0.0,  # value used for fill_mode = "constant"
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False,  # randomly flip images
    # set rescaling factor (applied before any other transformation)
    rescale=None,
    # set function that will be applied on each input
    preprocessing_function=None,
    # image data format, either "channels_first" or "channels_last"
    data_format="channels_last",
    # fraction of images reserved for validation (strictly between 0 and 1)
    validation_split=0.0,
)
image_generator = image_generator.flow_from_directory(
    directory='dataset/train',
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=64,
    class_mode="sparse",
    shuffle=True,
    seed=42
)

val_generator = ImageDataGenerator()
val_generator = val_generator.flow_from_directory(
    directory='dataset/val',
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=64,
    class_mode="sparse",
)

test_generator = ImageDataGenerator()
test_generator = test_generator.flow_from_directory(
    directory='dataset/test',
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=64,
    class_mode="sparse",
)

Found 29067 images belonging to 7 classes.
Found 3230 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.


In [4]:
# load the MobileNetV2 network, ensuring the head FC layer sets are left off
baseModel = MobileNetV2(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))
# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(len(label_map), activation="softmax")(headModel)
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
# for layer in baseModel.layers:
#     layer.trainable = False
    
optim = tf.keras.optimizers.Adam(learning_rate=1e-4)
model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'], optimizer=optim)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


/home/khuang/miniconda3/envs/DeepSpeech/lib/python3.6/site-packages/keras_applications/mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


In [9]:
checkpoint_path = Path('checkpoints/mobilenet/run2/')
checkpoint_path.mkdir(parents=True, exist_ok=True)
path_model='checkpoints/mobilenet/run2/{epoch:02d}-{val_loss:.6f}.hdf5'

h = model.fit(
    x=image_generator, 
    steps_per_epoch=len(image_generator),
    epochs=100, 
    verbose=1, 
    validation_data=val_generator,
    shuffle=True,
    use_multiprocessing=True,
    workers=4,
    callbacks=[
        ModelCheckpoint(filepath=path_model),
        EarlyStopping(patience=15),
        ReduceLROnPlateau(patience=6, factor=0.3)
    ]
)

Epoch 1/100
455/455 [==============================] - 333s 732ms/step - loss: 1.4614 - acc: 0.4412 - val_loss: 1.7206 - val_acc: 0.3607
Epoch 2/100
455/455 [==============================] - 286s 628ms/step - loss: 1.1556 - acc: 0.5680 - val_loss: 1.2893 - val_acc: 0.4997
Epoch 3/100
455/455 [==============================] - 283s 622ms/step - loss: 1.0501 - acc: 0.6106 - val_loss: 1.5491 - val_acc: 0.4567
Epoch 4/100
455/455 [==============================] - 283s 622ms/step - loss: 0.9741 - acc: 0.6389 - val_loss: 1.3522 - val_acc: 0.4808
Epoch 5/100
455/455 [==============================] - 283s 623ms/step - loss: 0.9140 - acc: 0.6661 - val_loss: 1.1138 - val_acc: 0.6065
Epoch 6/100
455/455 [==============================] - 282s 620ms/step - loss: 0.8573 - acc: 0.6839 - val_loss: 1.1219 - val_acc: 0.5898
Epoch 7/100
455/455 [==============================] - 282s 620ms/step - loss: 0.8110 - acc: 0.7057 - val_loss: 1.1651 - val_acc: 0.5944
Epoch 8/100
455/455 [====================

In [ ]:
im = Image.fromarray(np.uint8(X_train_224[1,:,:,:]*255))
im.save('test.jpg')

## Evaluate Model

In [24]:
expressionNet = load_model('checkpoints/mobilenet/run2/21-1.358947.hdf5')
scores = expressionNet.evaluate(test_generator, verbose=1)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
57/57 [==============================] - 8s 133ms/step - loss: 1.3393 - acc: 0.6888
